## 1. import

In [ ]:
import pandas as pd # V1.4.2
import numpy as np # V1.22.4

# sklearn V1.2.2
from sklearn.utils import shuffle 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split, cross_val_score,KFold,RepeatedKFold,GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# scipy V1.10.1
from scipy.stats import pearsonr, ttest_ind, levene
from scipy.stats import friedmanchisquare, rankdata

# matplotlib V3.5.2
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.pyplot import MultipleLocator
import matplotlib.ticker as ticker

import shap # V0.41.0 
# python 3.8

## 1.1 data split

In [ ]:
data = [...] # Data other than validations
# Stratified sampling to select test and training sets
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(test_size=0.25)
for train_index, test_index in split.split(data, data['label']):
    train_set = data.loc[train_index]
    test_set = data.loc[test_index] 

trains_AD = train_set['AD']

tests_AD = test_set['AD']

validation_AD = [...]

## 2. read data

### 2.1 GTV

In [ ]:
# Read the file
data0_filePath = "./data/features/gtv0_0519.csv"
data1_filePath = "./data/features/gtv1_0519.csv"
info_filePath = "./data/info_continuous_no-label_5.19.xlsx" # 临床特征
data0 = pd.read_csv(data0_filePath)
data1 = pd.read_csv(data1_filePath)
info = pd.read_excel(info_filePath)
rows0, cols0 = data0.shape
rows1, cols1 = data1.shape
print(rows0, cols0)
print(rows1, cols1)

# set lable
data0.insert(0,'label',[0] * rows0)
data1.insert(0,'label',[1] * rows1)
data = pd.concat([data0,data1])
data = shuffle(data)
data.index = range(len(data))

# split
train_gtv = data.loc[data['AD'].isin(trains_AD)]
test_gtv = data.loc[data['AD'].isin(tests_AD)]
validation_gtv = data.loc[data['AD'].isin(validation_AD)]

train_gtv.index = range(len(train_gtv))
test_gtv.index = range(len(test_gtv))
validation_gtv.index = range(len(validation_gtv))

print("train_gtv.shape: ", train_gtv.shape)
print("test_gtv.shape: ", test_gtv.shape)
print("validation_gtv.shape: ", validation_gtv.shape)

### 2.2 CTV

In [ ]:
ctv0_filePath = "./data/features/ctv0_0519.csv"
ctv1_filePath = "./data/features/ctv1_0519.csv"
ctv0 = pd.read_csv(ctv0_filePath)
ctv1 = pd.read_csv(ctv1_filePath)
rows0, cols0 = ctv0.shape
rows1, cols1 = ctv1.shape
print(rows0, cols0)
print(rows1, cols1)

# set lable
ctv0.insert(0,'label',[0] * rows0)
ctv1.insert(0,'label',[1] * rows1)
ctv = pd.concat([ctv0,ctv1])
ctv = shuffle(ctv)
ctv.index = range(len(ctv))

train_ctv = ctv.loc[ctv['AD'].isin(trains_AD)]
test_ctv = ctv.loc[ctv['AD'].isin(tests_AD)]
validation_ctv = data.loc[data['AD'].isin(validation_AD)]
train_ctv.index = range(len(train_ctv))
test_ctv.index = range(len(test_ctv))
validation_ctv.index = range(len(validation_ctv))

print("train_ctv.shape: ", train_ctv.shape)
print("test_ctv.shape: ", test_ctv.shape)
print("validation_ctv.shape: ", validation_ctv.shape)

### 2.3 Clinical

In [ ]:
# data 
info_filePath = "./data/info_continuous_5.19.xlsx"  # 临床特征
clinical = pd.read_excel(info_filePath)
clinical = shuffle(clinical)
clinical.index = range(len(clinical))

# split
train_clinical = clinical.loc[clinical['AD'].isin(trains_AD)]
test_clinical = clinical.loc[clinical['AD'].isin(tests_AD)]
validation_clinical = clinical.loc[clinical['AD'].isin(validation_AD)]

train_clinical.index = range(len(train_clinical))
test_clinical.index = range(len(test_clinical))
validation_clinical.index = range(len(validation_clinical))

print("train_clinical.shape: ", train_clinical.shape)
print("test_clinical.shape: ", test_clinical.shape)
print("validation_clinical.shape: ", validation_clinical.shape)

## 3. data with selected features
 (Features are derived from feature selection steps)

In [ ]:
info_MDL = info[["AD", "MDL"]]

### 3.1 GTV

In [ ]:
gtv_features = ['label', 'AD', 'original_shape_Elongation', 'original_shape_Flatness',
                'wavelet-HHH_glcm_Imc1', 'square_gldm_LargeDependenceHighGrayLevelEmphasis'] 
train_gtv = train_gtv[gtv_features]
test_gtv = test_gtv[gtv_features]
validation_gtv = validation_gtv[gtv_features]

### 3.2 GTV-Clinical

In [ ]:
train_gtv_clinical = pd.merge(train_gtv, info_MDL, on='AD')
test_gtv_clinical = pd.merge(test_gtv, info_MDL, on='AD')
validation_gtv_clinical = pd.merge(validation_gtv, info_MDL, on='AD')

gtv_clinical_features = ['label', 'AD', 'original_shape_Elongation', 'wavelet-HHH_glcm_Imc1',
                         'square_gldm_LargeDependenceHighGrayLevelEmphasis', 'MDL']
train_gtv_clinical = train_gtv_clinical[gtv_clinical_features]
test_gtv_clinical = test_gtv_clinical[gtv_clinical_features]
validation_gtv_clinical = validation_gtv_clinical[gtv_clinical_features]

### 3.3 CTV

In [ ]:
ctv_features = ['label', 'AD', 'wavelet-HLH_firstorder_Mean', 'wavelet-LLL_glcm_Imc1',
                'square_glcm_InverseVariance', 'gradient_firstorder_10Percentile']
train_ctv = train_ctv[ctv_features]
test_ctv = test_ctv[ctv_features]
validation_ctv = validation_ctv[ctv_features]

### 3.4 CTV-Clinical

In [ ]:
train_ctv_clinical = pd.merge(train_ctv, info_MDL, on='AD')
test_ctv_clinical = pd.merge(test_ctv, info_MDL, on='AD')
validation_ctv_clinical = pd.merge(validation_ctv, info_MDL, on='AD')

ctv_clinical_features = ['label', 'AD', 'wavelet-LLL_glcm_Imc1', 'wavelet-HLH_firstorder_Mean',
                         'square_glcm_InverseVariance', 'MDL']
train_ctv_clinical = train_ctv_clinical[ctv_clinical_features]
test_ctv_clinical = test_ctv_clinical[ctv_clinical_features]
validation_ctv_clinical = validation_ctv_clinical[ctv_clinical_features]

### 3.5 Clinical

In [ ]:
clinical_features = ["label","AD", "MDL"]
train_clinical = train_clinical[clinical_features]
test_clinical = test_clinical[clinical_features]
validation_clinical = validation_clinical[clinical_features]

### 3.6 TNM

In [ ]:
TNM_features = ["label", "AD", "T", "N"]
train_TNM = clinical.loc[clinical['AD'].isin(trains_AD)][TNM_features]
test_TNM = clinical.loc[clinical['AD'].isin(tests_AD)][TNM_features]
validation_TNM = clinical.loc[clinical['AD'].isin(validation_AD)][TNM_features]
cols_to_encode = ["T", "N"]
one_hot_encoded1 = pd.get_dummies(train_TNM[cols_to_encode])
one_hot_encoded2 = pd.get_dummies(test_TNM[cols_to_encode])
one_hot_encoded3 = pd.get_dummies(validation_TNM[cols_to_encode])

df_encoded1 = pd.concat([train_TNM, one_hot_encoded1], axis=1)
df_encoded2 = pd.concat([test_TNM, one_hot_encoded2], axis=1)
df_encoded3 = pd.concat([validation_TNM, one_hot_encoded3], axis=1)

train_TNM = df_encoded1.drop(cols_to_encode, axis=1)
test_TNM = df_encoded2.drop(cols_to_encode, axis=1)
validation_TNM = df_encoded3.drop(cols_to_encode, axis=1)

print("train_TNM.shape: ", train_TNM.shape)
print("test_TNM.shape: ", test_TNM.shape)
print("validation_TNM.shape: ", validation_TNM.shape)

## 4. ready for model

### 4.1 GTV

In [ ]:
train_set_gtv = train_gtv
test_set_gtv = test_gtv
validation_set_gtv = validation_gtv

In [ ]:
X_train_gtv = train_set_gtv[train_set_gtv.columns[2:]]
y_train_gtv = train_set_gtv['label']
X_train_gtv = X_train_gtv.apply(pd.to_numeric,errors = 'ignore') 
colNames = X_train_gtv.columns 
X_train_gtv = X_train_gtv.fillna(0)
X_train_gtv = X_train_gtv.astype(np.float64)
X_train_gtv = StandardScaler().fit_transform(X_train_gtv)
X_train_gtv = pd.DataFrame(X_train_gtv)
X_train_gtv.columns = colNames
print(X_train_gtv.shape)

X_test_gtv = test_set_gtv[test_set_gtv.columns[2:]]
y_test_gtv = test_set_gtv['label']
X_test_gtv = X_test_gtv.apply(pd.to_numeric,errors = 'ignore') 
colNames = X_test_gtv.columns 
X_test_gtv = X_test_gtv.fillna(0)
X_test_gtv = X_test_gtv.astype(np.float64)
X_test_gtv = StandardScaler().fit_transform(X_test_gtv)
X_test_gtv = pd.DataFrame(X_test_gtv)
X_test_gtv.columns = colNames
print(X_test_gtv.shape)

X_validation_gtv = validation_set_gtv[validation_set_gtv.columns[2:]]
y_validation_gtv = validation_set_gtv['label']
X_validation_gtv = X_validation_gtv.apply(pd.to_numeric,errors = 'ignore')
colNames = X_validation_gtv.columns 
X_validation_gtv = X_validation_gtv.astype(np.float64)
X_validation_gtv = StandardScaler().fit_transform(X_validation_gtv)
X_validation_gtv = pd.DataFrame(X_validation_gtv)
X_validation_gtv.columns = colNames
print(X_validation_gtv.shape)

### 4.2 GTV-Clinical

In [ ]:
train_set_gtv_clinical = train_gtv_clinical
test_set_gtv_clinical = test_gtv_clinical
validation_set_gtv_clinical = validation_gtv_clinical

In [ ]:
X_train_gtv_clinical = train_set_gtv_clinical[train_set_gtv_clinical.columns[2:]]
y_train_gtv_clinical = train_set_gtv_clinical['label']
X_train_gtv_clinical = X_train_gtv_clinical.apply(pd.to_numeric,errors = 'ignore') 
colNames = X_train_gtv_clinical.columns 
X_train_gtv_clinical = X_train_gtv_clinical.fillna(0)
X_train_gtv_clinical = X_train_gtv_clinical.astype(np.float64)
X_train_gtv_clinical = StandardScaler().fit_transform(X_train_gtv_clinical)
X_train_gtv_clinical = pd.DataFrame(X_train_gtv_clinical)
X_train_gtv_clinical.columns = colNames
print(X_train_gtv_clinical.shape)

X_test_gtv_clinical = test_set_gtv_clinical[test_set_gtv_clinical.columns[2:]]
y_test_gtv_clinical = test_set_gtv_clinical['label']
X_test_gtv_clinical = X_test_gtv_clinical.apply(pd.to_numeric,errors = 'ignore') 
colNames = X_test_gtv_clinical.columns 
X_test_gtv_clinical = X_test_gtv_clinical.fillna(0)
X_test_gtv_clinical = X_test_gtv_clinical.astype(np.float64)
X_test_gtv_clinical = StandardScaler().fit_transform(X_test_gtv_clinical)
X_test_gtv_clinical = pd.DataFrame(X_test_gtv_clinical)
X_test_gtv_clinical.columns = colNames
print(X_test_gtv_clinical.shape)

X_validation_gtv_clinical = validation_set_gtv_clinical[validation_set_gtv_clinical.columns[2:]]
y_validation_gtv_clinical = validation_set_gtv_clinical['label']
X_validation_gtv_clinical = X_validation_gtv_clinical.apply(pd.to_numeric,errors = 'ignore') 
colNames = X_validation_gtv_clinical.columns 
X_validation_gtv_clinical = X_validation_gtv_clinical.astype(np.float64)
X_validation_gtv_clinical = StandardScaler().fit_transform(X_validation_gtv_clinical)
X_validation_gtv_clinical = pd.DataFrame(X_validation_gtv_clinical)
X_validation_gtv_clinical.columns = colNames
print(X_validation_gtv_clinical.shape)

### 4.3 CTV

In [ ]:
train_set_ctv = train_ctv
test_set_ctv = train_ctv
validation_set_ctv = validation_ctv

In [ ]:
X_train_ctv = train_ctv[train_ctv.columns[2:]]
y_train_ctv = train_ctv['label']
X_train_ctv = X_train_ctv.apply(pd.to_numeric,errors = 'ignore') 
colNames = X_train_ctv.columns 
X_train_ctv = X_train_ctv.fillna(0)
X_train_ctv = X_train_ctv.astype(np.float64)
X_train_ctv = StandardScaler().fit_transform(X_train_ctv)
X_train_ctv = pd.DataFrame(X_train_ctv)
X_train_ctv.columns = colNames
print(X_train_ctv.shape)

X_test_ctv = test_ctv[test_ctv.columns[2:]]
y_test_ctv = test_ctv['label']
X_test_ctv = X_test_ctv.apply(pd.to_numeric,errors = 'ignore') 
colNames = X_test_ctv.columns
X_test_ctv = X_test_ctv.fillna(0)
X_test_ctv = X_test_ctv.astype(np.float64)
X_test_ctv = StandardScaler().fit_transform(X_test_ctv)
X_test_ctv = pd.DataFrame(X_test_ctv)
X_test_ctv.columns = colNames
print(X_test_ctv.shape)

X_validation_ctv = validation_set_ctv[validation_set_ctv.columns[2:]]
y_validation_ctv = validation_set_ctv['label']
X_validation_ctv = X_validation_ctv.apply(pd.to_numeric,errors = 'ignore') 
colNames = X_validation_ctv.columns 
X_validation_ctv = X_validation_ctv.astype(np.float64)
X_validation_ctv = StandardScaler().fit_transform(X_validation_ctv)
X_validation_ctv = pd.DataFrame(X_validation_ctv)
X_validation_ctv.columns = colNames
print(X_validation_ctv.shape)

### 4.4 CTV-Clinical

In [ ]:
train_set_ctv_clinical = train_ctv_clinical
test_set_ctv_clinical = test_ctv_clinical
validation_set_ctv_clinical = validation_ctv_clinical

In [ ]:
X_train_ctv_clinical = train_ctv_clinical[train_ctv_clinical.columns[2:]]
y_train_ctv_clinical = train_ctv_clinical['label']
X_train_ctv_clinical = X_train_ctv_clinical.apply(pd.to_numeric,errors = 'ignore') 
colNames = X_train_ctv_clinical.columns 
X_train_ctv_clinical = X_train_ctv_clinical.fillna(0)
X_train_ctv_clinical = X_train_ctv_clinical.astype(np.float64)
X_train_ctv_clinical = StandardScaler().fit_transform(X_train_ctv_clinical)
X_train_ctv_clinical = pd.DataFrame(X_train_ctv_clinical)
X_train_ctv_clinical.columns = colNames
print(X_train_ctv_clinical.shape)

X_test_ctv_clinical = test_ctv_clinical[test_ctv_clinical.columns[2:]]
y_test_ctv_clinical = test_ctv_clinical['label']
X_test_ctv_clinical = X_test_ctv_clinical.apply(pd.to_numeric,errors = 'ignore')
colNames = X_test_ctv_clinical.columns 
X_test_ctv_clinical = X_test_ctv_clinical.fillna(0)
X_test_ctv_clinical = X_test_ctv_clinical.astype(np.float64)
X_test_ctv_clinical = StandardScaler().fit_transform(X_test_ctv_clinical)
X_test_ctv_clinical = pd.DataFrame(X_test_ctv_clinical)
X_test_ctv_clinical.columns = colNames
print(X_test_ctv_clinical.shape)

X_validation_ctv_clinical = validation_set_ctv_clinical[validation_set_ctv_clinical.columns[2:]]
y_validation_ctv_clinical = validation_set_ctv_clinical['label']
X_validation_ctv_clinical = X_validation_ctv_clinical.apply(pd.to_numeric,errors = 'ignore')
colNames = X_validation_ctv_clinical.columns 
X_validation_ctv_clinical = X_validation_ctv_clinical.astype(np.float64)
X_validation_ctv_clinical = StandardScaler().fit_transform(X_validation_ctv_clinical)
X_validation_ctv_clinical = pd.DataFrame(X_validation_ctv_clinical)
X_validation_ctv_clinical.columns = colNames
print(X_validation_ctv_clinical.shape)

### 4.5 Clinical

In [ ]:
X_train_clinical = train_clinical[train_clinical.columns[2:]]
y_train_clinical = train_clinical['label']
X_train_clinical = X_train_clinical.apply(pd.to_numeric,errors = 'ignore') 
colNames = X_train_clinical.columns 
X_train_clinical = X_train_clinical.fillna(0)
X_train_clinical = X_train_clinical.astype(np.float64)
X_train_clinical = StandardScaler().fit_transform(X_train_clinical)
X_train_clinical = pd.DataFrame(X_train_clinical)
X_train_clinical.columns = colNames
print(X_train_clinical.shape)

X_test_clinical = test_clinical[test_clinical.columns[2:]]
y_test_clinical = test_clinical['label']
X_test_clinical = X_test_clinical.apply(pd.to_numeric,errors = 'ignore') 
colNames = X_test_clinical.columns 
X_test_clinical = X_test_clinical.fillna(0)
X_test_clinical = X_test_clinical.astype(np.float64)
X_test_clinical = StandardScaler().fit_transform(X_test_clinical)
X_test_clinical = pd.DataFrame(X_test_clinical)
X_test_clinical.columns = colNames
print(X_test_clinical.shape)

X_validation_clinical = validation_clinical[validation_clinical.columns[2:]]
y_validation_clinical = validation_clinical['label']
X_validation_clinical = X_validation_clinical.apply(pd.to_numeric,errors = 'ignore') 
colNames = X_validation_clinical.columns 
X_validation_clinical = X_validation_clinical.fillna(0)
X_validation_clinical = X_validation_clinical.astype(np.float64)
X_validation_clinical = StandardScaler().fit_transform(X_validation_clinical)
X_validation_clinical = pd.DataFrame(X_validation_clinical)
X_validation_clinical.columns = colNames
print(X_validation_clinical.shape)

### 4.6 TNM

In [ ]:
X_train_TNM = train_TNM[train_TNM.columns[2:]]
y_train_TNM = train_TNM['label']
colNames = X_train_TNM.columns 
X_train_TNM = X_train_TNM.fillna(0)
X_train_TNM = X_train_TNM.astype(np.float64)
X_train_TNM = StandardScaler().fit_transform(X_train_TNM)
X_train_TNM = pd.DataFrame(X_train_TNM)
X_train_TNM.columns = colNames
print(X_train_TNM.shape)

X_test_TNM = test_TNM[test_TNM.columns[2:]]
y_test_TNM = test_TNM['label']
colNames = X_test_TNM.columns 
X_test_TNM = X_test_TNM.fillna(0)
X_test_TNM = X_test_TNM.astype(np.float64)
X_test_TNM = StandardScaler().fit_transform(X_test_TNM)
X_test_TNM = pd.DataFrame(X_test_TNM)
X_test_TNM.columns = colNames
print(X_test_TNM.shape)

X_validation_TNM = validation_TNM[validation_TNM.columns[2:]]
y_validation_TNM = validation_TNM['label']
colNames = X_validation_TNM.columns 
X_validation_TNM = X_validation_TNM.fillna(0)
X_validation_TNM = X_validation_TNM.astype(np.float64)
X_validation_TNM = StandardScaler().fit_transform(X_validation_TNM)
X_validation_TNM = pd.DataFrame(X_validation_TNM)
X_validation_TNM.columns = colNames
print(X_validation_TNM.shape)


## 5. model

### 5.1 import

In [ ]:
from sklearn.metrics import precision_recall_curve,average_precision_score
from sklearn.metrics import f1_score,precision_score,recall_score,roc_auc_score,accuracy_score,roc_curve
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [ ]:
rs=2023

### 5.2 Adjustment of parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
#####################################################################################################
## gtv_clinical
score = []
for i in range(0,200,10):
    rfc = RandomForestClassifier(n_estimators=i+1,
                                 n_jobs=-1,
                                 random_state=rs)
    rfc.fit(X_train_gtv_clinical, y_train_gtv_clinical)
    y_pred = rfc.predict(X_test_gtv_clinical)
    score.append(round(accuracy_score(y_test_gtv_clinical,y_pred)*100,2))
    
rfc_n_estimators = score.index(max(score))*10
print(max(score),(score.index(max(score))*10))
plt.figure(figsize=(12, 6))  
plt.plot(range(1,201,10), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('The Learning curve')  
plt.xlabel('n_estimators Value')  
plt.ylabel('Score')

score = []
if rfc_n_estimators>5:
    for i in range(rfc_n_estimators-5,rfc_n_estimators+5):
        rfc = RandomForestClassifier(n_estimators=i,
                                     n_jobs=-1,
                                     random_state=rs)
        rfc.fit(X_train_gtv_clinical, y_train_gtv_clinical)
        y_pred = rfc.predict(X_test_gtv_clinical)
        score.append(round(accuracy_score(y_test_gtv_clinical,y_pred)*100,2))
    rfc_gtv_clinical= np.argmax(score)+rfc_n_estimators-5
else:
    for i in range(1,rfc_n_estimators+5):
        rfc = RandomForestClassifier(n_estimators=i,
                                     n_jobs=-1,
                                     random_state=rs)
        rfc.fit(X_train_gtv_clinical, y_train_gtv_clinical)
        y_pred = rfc.predict(X_test_gtv_clinical)
        score.append(round(accuracy_score(y_test_gtv_clinical,y_pred)*100,2))
    rfc_gtv_clinical= np.argmax(score)+1
    
print(rfc_gtv_clinical)
print(max(score))
plt.figure(figsize=(12, 6))  
if rfc_n_estimators>5:
    plt.plot(range(rfc_n_estimators-5,rfc_n_estimators+5), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
else:
    plt.plot(range(1,rfc_n_estimators+5), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('The Learning curve')  
plt.xlabel('n_estimators Value')  
plt.ylabel('Score')

rfc = RandomForestClassifier(n_estimators=rfc_gtv_clinical, random_state=rs)

# max_depth
param_grid = {'max_depth':np.arange(1,20)}
GS = GridSearchCV(rfc, param_grid, cv=10)
GS.fit(X_train_gtv_clinical, y_train_gtv_clinical)

best_param = GS.best_params_
best_score = GS.best_score_
print(best_param, best_score)
rfc_gtv_clinical_max_depth = best_param["max_depth"]

# max_features
param_grid = {'max_features':np.arange(1,31)}

rfc = RandomForestClassifier(n_estimators=rfc_gtv_clinical
                            ,random_state=rs
                            ,max_depth=rfc_gtv_clinical_max_depth)
GS = GridSearchCV(rfc, param_grid, cv=10)
GS.fit(X_train_gtv_clinical, y_train_gtv_clinical)
best_param = GS.best_params_
best_score = GS.best_score_
print(best_param, best_score) 

rfc_gtv_clinical_max_features = best_param["max_features"]
#####################################################################################################
## gtv
score = []
for i in range(0,200,10):
    rfc = RandomForestClassifier(n_estimators=i+1,
                                 n_jobs=-1,
                                 random_state=rs)
    rfc.fit(X_train_gtv, y_train_gtv)
    y_pred = rfc.predict(X_test_gtv)
    score.append(round(accuracy_score(y_test_gtv,y_pred)*100,2))
    
rfc_n_estimators = score.index(max(score))*10
print(max(score),rfc_n_estimators)
plt.figure(figsize=(12, 6))  
plt.plot(range(1,201,10), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('The Learning curve')  
plt.xlabel('n_estimators Value')  
plt.ylabel('Score')

score = []
if rfc_n_estimators>5:
    for i in range(rfc_n_estimators-5,rfc_n_estimators+5):
        rfc = RandomForestClassifier(n_estimators=i,
                                     n_jobs=-1,
                                     random_state=rs)
        rfc.fit(X_train_gtv, y_train_gtv)
        y_pred = rfc.predict(X_test_gtv)
        score.append(round(accuracy_score(y_test_gtv,y_pred)*100,2))
    rfc_gtv= np.argmax(score)+rfc_n_estimators-5
else:
    for i in range(1,rfc_n_estimators+5):
        rfc = RandomForestClassifier(n_estimators=i,
                                     n_jobs=-1,
                                     random_state=rs)
        rfc.fit(X_train_gtv, y_train_gtv)
        y_pred = rfc.predict(X_test_gtv)
        score.append(round(accuracy_score(y_test_gtv,y_pred)*100,2))
    rfc_gtv= np.argmax(score)+1
    
    
print(rfc_gtv)
print(max(score))
plt.figure(figsize=(12, 6))  
if rfc_n_estimators>5:
    plt.plot(range(rfc_n_estimators-5,rfc_n_estimators+5), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
else:
    plt.plot(range(1,rfc_n_estimators+5), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('The Learning curve')  
plt.xlabel('n_estimators Value')  
plt.ylabel('Score')

rfc = RandomForestClassifier(n_estimators=rfc_gtv, random_state=rs)

# max_depth
param_grid = {'max_depth':np.arange(1,20)}
GS = GridSearchCV(rfc, param_grid, cv=10)
GS.fit(X_train_gtv, y_train_gtv)

best_param = GS.best_params_
best_score = GS.best_score_
print(best_param, best_score)
rfc_gtv_max_depth = best_param["max_depth"]

# max_features
param_grid = {'max_features':np.arange(1,31)}

rfc = RandomForestClassifier(n_estimators=rfc_gtv
                            ,random_state=rs
                            ,max_depth=rfc_gtv_max_depth)
GS = GridSearchCV(rfc, param_grid, cv=10)
GS.fit(X_train_gtv, y_train_gtv)
best_param = GS.best_params_
best_score = GS.best_score_
print(best_param, best_score) 

rfc_gtv_max_features = best_param["max_features"]
#####################################################################################################
## ctv_clinical
score = []
for i in range(0,200,10):
    rfc = RandomForestClassifier(n_estimators=i+1,
                                 n_jobs=-1,
                                 random_state=rs)
    rfc.fit(X_train_ctv_clinical, y_train_ctv_clinical)
    y_pred = rfc.predict(X_test_ctv_clinical)
    score.append(round(accuracy_score(y_test_ctv_clinical,y_pred)*100,2))
    
rfc_n_estimators = score.index(max(score))*10
print(max(score),rfc_n_estimators)
plt.figure(figsize=(12, 6))  
plt.plot(range(1,201,10), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('The Learning curve')  
plt.xlabel('n_estimators Value')  
plt.ylabel('Score')

score = []
if rfc_n_estimators>5:
    for i in range(rfc_n_estimators-5,rfc_n_estimators+5):
        rfc = RandomForestClassifier(n_estimators=i,
                                     n_jobs=-1,
                                     random_state=rs)
        rfc.fit(X_train_ctv_clinical, y_train_ctv_clinical)
        y_pred = rfc.predict(X_test_ctv_clinical)
        score.append(round(accuracy_score(y_test_ctv_clinical,y_pred)*100,2))
    rfc_ctv_clinical= np.argmax(score)+rfc_n_estimators-5
else:
    for i in range(1,rfc_n_estimators+5):
        rfc = RandomForestClassifier(n_estimators=i,
                                     n_jobs=-1,
                                     random_state=rs)
        rfc.fit(X_train_ctv_clinical, y_train_ctv_clinical)
        y_pred = rfc.predict(X_test_ctv_clinical)
        score.append(round(accuracy_score(y_test_ctv_clinical,y_pred)*100,2))
    rfc_ctv_clinical= np.argmax(score)+1
    
print(rfc_ctv_clinical)
print(max(score))
plt.figure(figsize=(12, 6))  
if rfc_n_estimators>5:
    plt.plot(range(rfc_n_estimators-5,rfc_n_estimators+5), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
else:
    plt.plot(range(1,rfc_n_estimators+5), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('The Learning curve')  
plt.xlabel('n_estimators Value')  
plt.ylabel('Score')

rfc = RandomForestClassifier(n_estimators=rfc_ctv_clinical, random_state=rs)

# max_depth
param_grid = {'max_depth':np.arange(1,20)}
GS = GridSearchCV(rfc, param_grid, cv=10)
GS.fit(X_train_ctv_clinical, y_train_ctv_clinical)

best_param = GS.best_params_
best_score = GS.best_score_
print(best_param, best_score)
rfc_ctv_clinical_max_depth = best_param["max_depth"]

# max_features
param_grid = {'max_features':np.arange(1,31)}

rfc = RandomForestClassifier(n_estimators=rfc_ctv_clinical
                            ,random_state=rs
                            ,max_depth=rfc_ctv_clinical_max_depth)
GS = GridSearchCV(rfc, param_grid, cv=10)
GS.fit(X_train_ctv_clinical, y_train_ctv_clinical)
best_param = GS.best_params_
best_score = GS.best_score_
print(best_param, best_score) 

rfc_ctv_clinical_max_features = best_param["max_features"]
#####################################################################################################
## ctv
score = []
for i in range(0,200,10):
    rfc = RandomForestClassifier(n_estimators=i+1,
                                 n_jobs=-1,
                                 random_state=rs)
    rfc.fit(X_train_ctv, y_train_ctv)
    y_pred = rfc.predict(X_test_ctv)
    score.append(round(accuracy_score(y_test_ctv,y_pred)*100,2))
    
rfc_n_estimators = score.index(max(score))*10
print(max(score),rfc_n_estimators)
plt.figure(figsize=(12, 6))  
plt.plot(range(1,201,10), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('The Learning curve')  
plt.xlabel('n_estimators Value')  
plt.ylabel('Score')

score = []
if rfc_n_estimators>5:
    for i in range(rfc_n_estimators-5,rfc_n_estimators+5):
        rfc = RandomForestClassifier(n_estimators=i,
                                 n_jobs=-1,
                                 random_state=rs)
        rfc.fit(X_train_ctv, y_train_ctv)
        y_pred = rfc.predict(X_test_ctv)
        score.append(round(accuracy_score(y_test_ctv,y_pred)*100,2))
    rfc_ctv= np.argmax(score)+rfc_n_estimators-5
else:
    for i in range(1,rfc_n_estimators+5):
        rfc = RandomForestClassifier(n_estimators=i,
                                     n_jobs=-1,
                                     random_state=rs)
        rfc.fit(X_train_ctv, y_train_ctv)
        y_pred = rfc.predict(X_test_ctv)
        score.append(round(accuracy_score(y_test_ctv,y_pred)*100,2))
    rfc_ctv= np.argmax(score)+1

print(rfc_ctv)
print(max(score))
plt.figure(figsize=(12, 6))  
if rfc_n_estimators>5:
    plt.plot(range(rfc_n_estimators-5,rfc_n_estimators+5), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
else:
    plt.plot(range(1,rfc_n_estimators+5), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)

plt.title('The Learning curve')  
plt.xlabel('n_estimators Value')  
plt.ylabel('Score')

rfc = RandomForestClassifier(n_estimators=rfc_ctv, random_state=rs)

# 用网格搜索调整max_depth
param_grid = {'max_depth':np.arange(1,20)}
GS = GridSearchCV(rfc, param_grid, cv=10)
GS.fit(X_train_ctv, y_train_ctv)

best_param = GS.best_params_
best_score = GS.best_score_
print(best_param, best_score)
rfc_ctv_max_depth = best_param["max_depth"]

# 用网格搜索调整max_features
param_grid = {'max_features':np.arange(1,31)}

rfc = RandomForestClassifier(n_estimators=rfc_ctv
                            ,random_state=rs
                            ,max_depth=rfc_ctv_max_depth)
GS = GridSearchCV(rfc, param_grid, cv=10)
GS.fit(X_train_ctv, y_train_ctv)
best_param = GS.best_params_
best_score = GS.best_score_
print(best_param, best_score) 

rfc_ctv_max_features = best_param["max_features"]
#####################################################################################################
## clinical
score = []
for i in range(0,200,10):
    rfc = RandomForestClassifier(n_estimators=i+1,
                                 n_jobs=-1,
                                 random_state=rs)
    rfc.fit(X_train_clinical, y_train_clinical)
    y_pred = rfc.predict(X_test_clinical)
    score.append(round(accuracy_score(y_test_clinical,y_pred)*100,2))
    
rfc_n_estimators = score.index(max(score))*10
print(max(score),rfc_n_estimators)
plt.figure(figsize=(12, 6))  
plt.plot(range(1,201,10), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('The Learning curve')  
plt.xlabel('n_estimators Value')  
plt.ylabel('Score')

score = []
if rfc_n_estimators>5:
    for i in range(rfc_n_estimators-5,rfc_n_estimators+5):
        rfc = RandomForestClassifier(n_estimators=i,
                                     n_jobs=-1,
                                     random_state=rs)
        rfc.fit(X_train_clinical, y_train_clinical)
        y_pred = rfc.predict(X_test_clinical)
        score.append(round(accuracy_score(y_test_clinical,y_pred)*100,2))
    rfc_clinical= np.argmax(score)+rfc_n_estimators-5
else:
    for i in range(1,rfc_n_estimators+5):
        rfc = RandomForestClassifier(n_estimators=i,
                                     n_jobs=-1,
                                     random_state=rs)
        rfc.fit(X_train_clinical, y_train_clinical)
        y_pred = rfc.predict(X_test_clinical)
        score.append(round(accuracy_score(y_test_clinical,y_pred)*100,2))
    rfc_clinical = np.argmax(score)+1
        

print(rfc_clinical)
print(max(score))
plt.figure(figsize=(12, 6))  
if rfc_n_estimators>5:
    plt.plot(range(rfc_n_estimators-5,rfc_n_estimators+5), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
else:
    plt.plot(range(1,rfc_n_estimators+5), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('The Learning curve')  
plt.xlabel('n_estimators Value')  
plt.ylabel('Score')

rfc = RandomForestClassifier(n_estimators=rfc_clinical, random_state=rs)

# 用网格搜索调整max_depth
param_grid = {'max_depth':np.arange(1,20)}
GS = GridSearchCV(rfc, param_grid, cv=10)
GS.fit(X_train_clinical, y_train_clinical)

best_param = GS.best_params_
best_score = GS.best_score_
print(best_param, best_score)
rfc_clinical_max_depth = best_param["max_depth"]

# 用网格搜索调整max_features
param_grid = {'max_features':np.arange(1,31)}

rfc = RandomForestClassifier(n_estimators=rfc_clinical
                            ,random_state=rs
                            ,max_depth=rfc_clinical_max_depth)
GS = GridSearchCV(rfc, param_grid, cv=10)
GS.fit(X_train_clinical, y_train_clinical)
best_param = GS.best_params_
best_score = GS.best_score_
print(best_param, best_score) 

rfc_clinical_max_features = best_param["max_features"]

#####################################################################################################
## TNM
score = []
for i in range(0,200,10):
    rfc = RandomForestClassifier(n_estimators=i+1,
                                 n_jobs=-1,
                                 random_state=rs)
    rfc.fit(X_train_TNM, y_train_TNM)
    y_pred = rfc.predict(X_test_TNM)
    score.append(round(accuracy_score(y_test_TNM,y_pred)*100,2))
    
rfc_n_estimators = score.index(max(score))*10
print(max(score),rfc_n_estimators)
plt.figure(figsize=(12, 6))  
plt.plot(range(1,201,10), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('The Learning curve')  
plt.xlabel('n_estimators Value')  
plt.ylabel('Score')

score = []
if rfc_n_estimators>5:
    for i in range(rfc_n_estimators-5,rfc_n_estimators+5):
        rfc = RandomForestClassifier(n_estimators=i,
                                     n_jobs=-1,
                                     random_state=rs)
        rfc.fit(X_train_TNM, y_train_TNM)
        y_pred = rfc.predict(X_test_TNM)
        score.append(round(accuracy_score(y_test_TNM,y_pred)*100,2))
    rfc_TNM= np.argmax(score)+rfc_n_estimators-5
else:
    for i in range(1,rfc_n_estimators+5):
        rfc = RandomForestClassifier(n_estimators=i,
                                     n_jobs=-1,
                                     random_state=rs)
        rfc.fit(X_train_TNM, y_train_TNM)
        y_pred = rfc.predict(X_test_TNM)
        score.append(round(accuracy_score(y_test_TNM,y_pred)*100,2))
    rfc_TNM = np.argmax(score)+1
        

print(rfc_TNM)
print(max(score))
plt.figure(figsize=(12, 6))  
if rfc_n_estimators>5:
    plt.plot(range(rfc_n_estimators-5,rfc_n_estimators+5), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
else:
    plt.plot(range(1,rfc_n_estimators+5), score, color='red', linestyle='dashed', marker='o',  
         markerfacecolor='blue', markersize=10)
plt.title('The Learning curve')  
plt.xlabel('n_estimators Value')  
plt.ylabel('Score')

rfc = RandomForestClassifier(n_estimators=rfc_TNM, random_state=rs)

# 用网格搜索调整max_depth
param_grid = {'max_depth':np.arange(1,20)}
GS = GridSearchCV(rfc, param_grid, cv=10)
GS.fit(X_train_TNM, y_train_TNM)

best_param = GS.best_params_
best_score = GS.best_score_
print(best_param, best_score)
rfc_TNM_max_depth = best_param["max_depth"]

# 用网格搜索调整max_features
param_grid = {'max_features':np.arange(1,31)}

rfc = RandomForestClassifier(n_estimators=rfc_TNM
                            ,random_state=rs
                            ,max_depth=rfc_TNM_max_depth)
GS = GridSearchCV(rfc, param_grid, cv=10)
GS.fit(X_train_TNM, y_train_TNM)
best_param = GS.best_params_
best_score = GS.best_score_
print(best_param, best_score) 

rfc_TNM_max_features = best_param["max_features"]


### 5.3 训练集建模

In [ ]:
## GTV-CLINI
print("GTV-clinical")
# Constructing  model based on the parameters
forest_gtv_clinical=RandomForestClassifier(n_estimators=rfc_gtv_clinical
                            ,random_state=rs
                            ,max_depth=rfc_gtv_clinical_max_depth
                            ,max_features=rfc_gtv_clinical_max_features) 

forest_gtv_clinical.fit(X_train_gtv_clinical, y_train_gtv_clinical)


forest_y_pre=forest_gtv_clinical.predict(X_test_gtv_clinical)
forest_y_proba_gtv_clinical=forest_gtv_clinical.predict_proba(X_test_gtv_clinical)

forest_accuracy_score=accuracy_score(y_test_gtv_clinical,forest_y_pre)
forest_preci_score_gtv_clinical=precision_score(y_test_gtv_clinical,forest_y_pre)
forest_recall_score_gtv_clinical=recall_score(y_test_gtv_clinical,forest_y_pre)
forest_f1_score_gtv_clinical=f1_score(y_test_gtv_clinical,forest_y_pre)
forest_auc_gtv_clinical=roc_auc_score(y_test_gtv_clinical,forest_y_proba_gtv_clinical[:,1])

precision_gtv_clinical, recall_gtv_clinical, _ = precision_recall_curve(y_test_gtv_clinical,forest_y_pre)
PRC_gtv_clinical = average_precision_score(y_test_gtv_clinical,forest_y_pre)

print('forest_accuracy_score: %f,forest_preci_score: %f,forest_recall_score: %f,forest_f1_score: %f,forest_auc_gtv_clinical: %f'
      %(forest_accuracy_score,forest_preci_score_gtv_clinical,forest_recall_score_gtv_clinical,forest_f1_score_gtv_clinical,forest_auc_gtv_clinical))



forest_y_pre_train = forest_gtv_clinical.predict(X_train_gtv_clinical)
forest_y_proba_gtv_clinical_train = forest_gtv_clinical.predict_proba(X_train_gtv_clinical)

forest_accuracy_score_train = accuracy_score(y_train_gtv_clinical,forest_y_pre_train)
forest_preci_score_gtv_clinical_train = precision_score(y_train_gtv_clinical,forest_y_pre_train)
forest_recall_score_gtv_clinical_train = recall_score(y_train_gtv_clinical,forest_y_pre_train)
forest_f1_score_gtv_clinical_train = f1_score(y_train_gtv_clinical,forest_y_pre_train)
forest_auc_gtv_clinical_train = roc_auc_score(y_train_gtv_clinical,forest_y_proba_gtv_clinical_train[:,1])

precision_gtv_clinical_train, recall_gtv_clinical_train, _ = precision_recall_curve(y_train_gtv_clinical,forest_y_pre_train)
PRC_gtv_clinical_train = average_precision_score(y_train_gtv_clinical,forest_y_pre_train)

print('forest_accuracy_score_train: %f,forest_preci_score_train: %f,forest_recall_score_train: %f,forest_f1_score_train: %f,forest_auc_gtv_clinical_train: %f'
      %(forest_accuracy_score_train,forest_preci_score_gtv_clinical_train,forest_recall_score_gtv_clinical_train,forest_f1_score_gtv_clinical_train,forest_auc_gtv_clinical_train))


forest_y_pre_validation = forest_gtv_clinical.predict(X_validation_gtv_clinical)
forest_y_proba_gtv_clinical_validation = forest_gtv_clinical.predict_proba(X_validation_gtv_clinical)

forest_accuracy_score_validation = accuracy_score(y_validation_gtv_clinical,forest_y_pre_validation)
forest_preci_score_gtv_clinical_validation = precision_score(y_validation_gtv_clinical,forest_y_pre_validation)
forest_recall_score_gtv_clinical_validation = recall_score(y_validation_gtv_clinical,forest_y_pre_validation)
forest_f1_score_gtv_clinical_validation = f1_score(y_validation_gtv_clinical,forest_y_pre_validation)
forest_auc_gtv_clinical_validation = roc_auc_score(y_validation_gtv_clinical,forest_y_proba_gtv_clinical_validation[:,1])

precision_gtv_clinical_validation, recall_gtv_clinical_validation, _ = precision_recall_curve(y_validation_gtv_clinical,forest_y_pre_validation)
PRC_gtv_clinical_validation = average_precision_score(y_validation_gtv_clinical,forest_y_pre_validation)

print('forest_accuracy_score_validation: %f,forest_preci_score_validation: %f,forest_recall_score_validation: %f,forest_f1_score_validation: %f,forest_auc_gtv_clinical_validation: %f'
      %(forest_accuracy_score_validation,forest_preci_score_gtv_clinical_validation,forest_recall_score_gtv_clinical_validation,forest_f1_score_gtv_clinical_validation,forest_auc_gtv_clinical_validation))

print('-----------------------------------------------------------------------','\n')

#############################################################################################################################################
## GTV
print("GTV")
forest_gtv=RandomForestClassifier(n_estimators=rfc_gtv
                            ,random_state=rs
                            ,max_depth=rfc_gtv_max_depth
                            ,max_features=rfc_gtv_max_features) 

forest_gtv.fit(X_train_gtv, y_train_gtv)

forest_y_pre=forest_gtv.predict(X_test_gtv)
forest_y_proba_gtv=forest_gtv.predict_proba(X_test_gtv)

forest_accuracy_score = accuracy_score(y_test_gtv,forest_y_pre)
forest_preci_score_gtv = precision_score(y_test_gtv,forest_y_pre)
forest_recall_score_gtv = recall_score(y_test_gtv,forest_y_pre)
forest_f1_score_gtv = f1_score(y_test_gtv,forest_y_pre)
forest_auc_gtv = roc_auc_score(y_test_gtv,forest_y_proba_gtv[:,1])

precision_gtv, recall_gtv, _ = precision_recall_curve(y_test_gtv,forest_y_pre)
PRC_gtv = average_precision_score(y_test_gtv,forest_y_pre)

print('forest_accuracy_score: %f,forest_preci_score: %f,forest_recall_score: %f,forest_f1_score: %f,forest_auc_gtv: %f'
      %(forest_accuracy_score,forest_preci_score_gtv,forest_recall_score_gtv,forest_f1_score_gtv,forest_auc_gtv))

# train
forest_y_pre_train = forest_gtv.predict(X_train_gtv)
forest_y_proba_gtv_train = forest_gtv.predict_proba(X_train_gtv)

forest_accuracy_score_train = accuracy_score(y_train_gtv,forest_y_pre_train)
forest_preci_score_gtv_train = precision_score(y_train_gtv,forest_y_pre_train)
forest_recall_score_gtv_train = recall_score(y_train_gtv,forest_y_pre_train)
forest_f1_score_gtv_train = f1_score(y_train_gtv,forest_y_pre_train)
forest_auc_gtv_train = roc_auc_score(y_train_gtv,forest_y_proba_gtv_train[:,1])

precision_gtv_train, recall_gtvv, _ = precision_recall_curve(y_train_gtv,forest_y_pre_train)
PRC_gtv_train = average_precision_score(y_train_gtv,forest_y_pre_train)

print('forest_accuracy_score_train: %f,forest_preci_score_train: %f,forest_recall_score_train: %f,forest_f1_score_train: %f,forest_auc_gtv_train: %f'
      %(forest_accuracy_score_train,forest_preci_score_gtv_train,forest_recall_score_gtv_train,forest_f1_score_gtv_train,forest_auc_gtv_train))

# validation
forest_y_pre_validation = forest_gtv.predict(X_validation_gtv)
forest_y_proba_gtv_validation = forest_gtv.predict_proba(X_validation_gtv)

forest_accuracy_score_validation = accuracy_score(y_validation_gtv,forest_y_pre_validation)
forest_preci_score_gtv_validation = precision_score(y_validation_gtv,forest_y_pre_validation)
forest_recall_score_gtv_validation = recall_score(y_validation_gtv,forest_y_pre_validation)
forest_f1_score_gtv_validation = f1_score(y_validation_gtv,forest_y_pre_validation)
forest_auc_gtv_validation = roc_auc_score(y_validation_gtv,forest_y_proba_gtv_validation[:,1])

precision_gtv_validation, recall_gtvv, _ = precision_recall_curve(y_validation_gtv,forest_y_pre_validation)
PRC_gtv_validation = average_precision_score(y_validation_gtv,forest_y_pre_validation)

print('forest_accuracy_score_validation: %f,forest_preci_score_validation: %f,forest_recall_score_validation: %f,forest_f1_score_validation: %f,forest_auc_gtv_validation: %f'
      %(forest_accuracy_score_validation,forest_preci_score_gtv_validation,forest_recall_score_gtv_validation,forest_f1_score_gtv_validation,forest_auc_gtv_validation))

print('-----------------------------------------------------------------------','\n')

#############################################################################################################################################
## CTV-CLINICAL
print("CTV-CLINICAL")
forest_ctv_clinical=RandomForestClassifier(n_estimators=rfc_ctv_clinical
                            ,random_state=rs
                            ,max_depth=rfc_ctv_clinical_max_depth
                            ,max_features=rfc_ctv_clinical_max_features) 

forest_ctv_clinical.fit(X_train_ctv_clinical, y_train_ctv_clinical)

forest_y_pre=forest_ctv_clinical.predict(X_test_ctv_clinical)
forest_y_proba_ctv_clinical=forest_ctv_clinical.predict_proba(X_test_ctv_clinical)

forest_accuracy_score = accuracy_score(y_test_ctv_clinical,forest_y_pre)
forest_preci_score_ctv_clinical = precision_score(y_test_ctv_clinical,forest_y_pre)
forest_recall_score_ctv_clinical = recall_score(y_test_ctv_clinical,forest_y_pre)
forest_f1_score_ctv_clinical = f1_score(y_test_ctv_clinical,forest_y_pre)
forest_auc_ctv_clinical=roc_auc_score(y_test_ctv_clinical,forest_y_proba_ctv_clinical[:,1])


precision_ctv_clinical, recall_ctv_clinical, _ = precision_recall_curve(y_test_ctv_clinical,forest_y_pre)
PRC_ctv_clinical = average_precision_score(y_test_ctv_clinical,forest_y_pre)

print('forest_accuracy_score: %f,forest_preci_score: %f,forest_recall_score: %f,forest_f1_score: %f,forest_auc_ctv_clinical: %f'
      %(forest_accuracy_score,forest_preci_score_ctv_clinical,forest_recall_score_ctv_clinical,forest_f1_score_ctv_clinical,forest_auc_ctv_clinical))

# train
forest_y_pre_train = forest_ctv_clinical.predict(X_train_ctv_clinical)
forest_y_proba_ctv_clinical_train = forest_ctv_clinical.predict_proba(X_train_ctv_clinical)

forest_accuracy_score_train = accuracy_score(y_train_ctv_clinical,forest_y_pre_train)
forest_preci_score_ctv_clinical_train = precision_score(y_train_ctv_clinical,forest_y_pre_train)
forest_recall_score_ctv_clinical_train = recall_score(y_train_ctv_clinical,forest_y_pre_train)
forest_f1_score_ctv_clinical_train = f1_score(y_train_ctv_clinical,forest_y_pre_train)
forest_auc_ctv_clinical_train = roc_auc_score(y_train_ctv_clinical,forest_y_proba_ctv_clinical_train[:,1])


precision_ctv_clinical_train, recall_ctv_clinical_train, _ = precision_recall_curve(y_train_ctv_clinical,forest_y_pre_train)
PRC_ctv_clinical_train = average_precision_score(y_train_ctv_clinical,forest_y_pre_train)

print('forest_accuracy_score_train: %f,forest_preci_score_train: %f,forest_recall_score_train: %f,forest_f1_score_train: %f,forest_auc_ctv_clinical_train: %f'
      %(forest_accuracy_score_train,forest_preci_score_ctv_clinical_train,forest_recall_score_ctv_clinical_train,forest_f1_score_ctv_clinical_train,forest_auc_ctv_clinical_train))

# validation
forest_y_pre_validation = forest_ctv_clinical.predict(X_validation_ctv_clinical)
forest_y_proba_ctv_clinical_validation = forest_ctv_clinical.predict_proba(X_validation_ctv_clinical)

forest_accuracy_score_validation = accuracy_score(y_validation_ctv_clinical,forest_y_pre_validation)
forest_preci_score_ctv_clinical_validation = precision_score(y_validation_ctv_clinical,forest_y_pre_validation)
forest_recall_score_ctv_clinical_validation = recall_score(y_validation_ctv_clinical,forest_y_pre_validation)
forest_f1_score_ctv_clinical_validation = f1_score(y_validation_ctv_clinical,forest_y_pre_validation)
forest_auc_ctv_clinical_validation = roc_auc_score(y_validation_ctv_clinical,forest_y_proba_ctv_clinical_validation[:,1])


precision_ctv_clinical_validation, recall_ctv_clinical_validation, _ = precision_recall_curve(y_validation_ctv_clinical,forest_y_pre_validation)
PRC_ctv_clinical_validation = average_precision_score(y_validation_ctv_clinical,forest_y_pre_validation)

print('forest_accuracy_score_validation: %f,forest_preci_score_validation: %f,forest_recall_score_validation: %f,forest_f1_score_validation: %f,forest_auc_ctv_clinical_validation: %f'
      %(forest_accuracy_score_validation,forest_preci_score_ctv_clinical_validation,forest_recall_score_ctv_clinical_validation,forest_f1_score_ctv_clinical_validation,forest_auc_ctv_clinical_validation))

print('-----------------------------------------------------------------------','\n')

#############################################################################################################################################
## CTV
print("CTV")
forest_ctv=RandomForestClassifier(n_estimators=rfc_ctv
                            ,random_state=rs
                            ,max_depth=rfc_ctv_max_depth
                            ,max_features=rfc_ctv_max_features) 

forest_ctv.fit(X_train_ctv, y_train_ctv)

forest_y_pre=forest_ctv.predict(X_test_ctv)
forest_y_proba_ctv=forest_ctv.predict_proba(X_test_ctv)

forest_accuracy_score=accuracy_score(y_test_ctv,forest_y_pre)
forest_preci_score_ctv = precision_score(y_test_ctv,forest_y_pre)
forest_recall_score_ctv = recall_score(y_test_ctv,forest_y_pre)
forest_f1_score_ctv = f1_score(y_test_ctv,forest_y_pre)
forest_auc_ctv=roc_auc_score(y_test_ctv,forest_y_proba_ctv[:,1])

precision_ctv, recall_ctv, _ = precision_recall_curve(y_test_ctv,forest_y_pre)
PRC_ctv = average_precision_score(y_test_ctv,forest_y_pre)

print('forest_accuracy_score: %f,forest_preci_score: %f,forest_recall_score: %f,forest_f1_score: %f,forest_auc_ctv: %f'
      %(forest_accuracy_score,forest_preci_score_ctv,forest_recall_score_ctv,forest_f1_score_ctv,forest_auc_ctv))

# train
forest_y_pre_train = forest_ctv.predict(X_train_ctv)
forest_y_proba_ctv_train = forest_ctv.predict_proba(X_train_ctv)

forest_accuracy_score_train = accuracy_score(y_train_ctv,forest_y_pre_train)
forest_preci_score_ctv_train = precision_score(y_train_ctv,forest_y_pre_train)
forest_recall_score_ctv_train = recall_score(y_train_ctv,forest_y_pre_train)
forest_f1_score_ctv_train = f1_score(y_train_ctv,forest_y_pre_train)
forest_auc_ctv_train = roc_auc_score(y_train_ctv,forest_y_proba_ctv_train[:,1])

precision_ctv_train, recall_ctv_train, _ = precision_recall_curve(y_train_ctv,forest_y_pre_train)
PRC_ctv_train = average_precision_score(y_train_ctv,forest_y_pre_train)

print('forest_accuracy_score_train: %f,forest_preci_score_train: %f,forest_recall_score_train: %f,forest_f1_score_train: %f,forest_auc_ctv_train: %f'
      %(forest_accuracy_score_train,forest_preci_score_ctv_train,forest_recall_score_ctv_train,forest_f1_score_ctv_train,forest_auc_ctv_train))

# validation
forest_y_pre_validation = forest_ctv.predict(X_validation_ctv)
forest_y_proba_ctv_validation = forest_ctv.predict_proba(X_validation_ctv)

forest_accuracy_score_validation = accuracy_score(y_validation_ctv,forest_y_pre_validation)
forest_preci_score_ctv_validation = precision_score(y_validation_ctv,forest_y_pre_validation)
forest_recall_score_ctv_validation = recall_score(y_validation_ctv,forest_y_pre_validation)
forest_f1_score_ctv_validation = f1_score(y_validation_ctv,forest_y_pre_validation)
forest_auc_ctv_validation = roc_auc_score(y_validation_ctv,forest_y_proba_ctv_validation[:,1])

precision_ctv_validation, recall_ctv_validation, _ = precision_recall_curve(y_validation_ctv,forest_y_pre_validation)
PRC_ctv_validation = average_precision_score(y_validation_ctv,forest_y_pre_validation)

print('forest_accuracy_score_validation: %f,forest_preci_score_validation: %f,forest_recall_score_validation: %f,forest_f1_score_validation: %f,forest_auc_ctv_validation: %f'
      %(forest_accuracy_score_validation,forest_preci_score_ctv_validation,forest_recall_score_ctv_validation,forest_f1_score_ctv_validation,forest_auc_ctv_validation))

print('-----------------------------------------------------------------------','\n')

#############################################################################################################################################
## CLINICAL
print("CLINICAL")
forest_clinical = RandomForestClassifier(n_estimators=rfc_clinical
                            ,random_state=rs
                            ,max_depth=rfc_clinical_max_depth
                            ,max_features=rfc_clinical_max_features) 

forest_clinical.fit(X_train_clinical, y_train_clinical)

forest_y_pre=forest_clinical.predict(X_test_clinical)
forest_y_proba_clinical=forest_clinical.predict_proba(X_test_clinical)

forest_accuracy_score=accuracy_score(y_test_clinical,forest_y_pre)
forest_preci_score_clinical = precision_score(y_test_clinical,forest_y_pre)
forest_recall_score_clinical = recall_score(y_test_clinical,forest_y_pre)
forest_f1_score_clinical = f1_score(y_test_clinical,forest_y_pre)
forest_auc_clinical=roc_auc_score(y_test_clinical,forest_y_proba_clinical[:,1])

precision_clinical, recall_clinical, _ = precision_recall_curve(y_test_clinical,forest_y_pre)
PRC_clinical = average_precision_score(y_test_clinical,forest_y_pre)

print('forest_accuracy_score: %f,forest_preci_score: %f,forest_recall_score: %f,forest_f1_score: %f,forest_auc_clinical: %f'
      %(forest_accuracy_score,forest_preci_score_clinical,forest_recall_score_clinical,forest_f1_score_clinical,forest_auc_clinical))

# train
forest_y_pre_train=forest_clinical.predict(X_train_clinical)
forest_y_proba_clinical_train=forest_clinical.predict_proba(X_train_clinical)

forest_accuracy_score_train = accuracy_score(y_train_clinical,forest_y_pre_train)
forest_preci_score_clinical_train = precision_score(y_train_clinical,forest_y_pre_train)
forest_recall_score_clinical_train = recall_score(y_train_clinical,forest_y_pre_train)
forest_f1_score_clinical_train = f1_score(y_train_clinical,forest_y_pre_train)
forest_auc_clinical_train = roc_auc_score(y_train_clinical,forest_y_proba_clinical_train[:,1])

precision_clinical_train, recall_clinical_train, _ = precision_recall_curve(y_train_clinical,forest_y_pre_train)
PRC_clinical_train = average_precision_score(y_train_clinical,forest_y_pre_train)

print('forest_accuracy_score_train: %f,forest_preci_score_train: %f,forest_recall_score_train: %f,forest_f1_score_train: %f,forest_auc_clinical_train: %f'
      %(forest_accuracy_score_train,forest_preci_score_clinical_train,forest_recall_score_clinical_train,forest_f1_score_clinical_train,forest_auc_clinical_train))

# validation
forest_y_pre_validation=forest_clinical.predict(X_validation_clinical)
forest_y_proba_clinical_validation=forest_clinical.predict_proba(X_validation_clinical)

forest_accuracy_score_validation = accuracy_score(y_validation_clinical,forest_y_pre_validation)
forest_preci_score_clinical_validation = precision_score(y_validation_clinical,forest_y_pre_validation)
forest_recall_score_clinical_validation = recall_score(y_validation_clinical,forest_y_pre_validation)
forest_f1_score_clinical_validation = f1_score(y_validation_clinical,forest_y_pre_validation)
forest_auc_clinical_validation = roc_auc_score(y_validation_clinical,forest_y_proba_clinical_validation[:,1])

precision_clinical_validation, recall_clinical_validation, _ = precision_recall_curve(y_validation_clinical,forest_y_pre_validation)
PRC_clinical_validation = average_precision_score(y_validation_clinical,forest_y_pre_validation)

print('forest_accuracy_score_validation: %f,forest_preci_score_validation: %f,forest_recall_score_validation: %f,forest_f1_score_validation: %f,forest_auc_clinical_validation: %f'
      %(forest_accuracy_score_validation,forest_preci_score_clinical_validation,forest_recall_score_clinical_validation,forest_f1_score_clinical_validation,forest_auc_clinical_validation))


print('-----------------------------------------------------------------------','\n')

#############################################################################################################################################
## TNM
print("TNM")
forest_TNM = RandomForestClassifier(n_estimators=rfc_TNM
                            ,random_state=rs
                            ,max_depth=rfc_TNM_max_depth
                            ,max_features=rfc_TNM_max_features) 

forest_TNM.fit(X_train_TNM, y_train_TNM)

forest_y_pre=forest_TNM.predict(X_test_TNM)
forest_y_proba_TNM=forest_TNM.predict_proba(X_test_TNM)

forest_accuracy_score=accuracy_score(y_test_TNM,forest_y_pre)
forest_preci_score_TNM = precision_score(y_test_TNM,forest_y_pre)
forest_recall_score_TNM = recall_score(y_test_TNM,forest_y_pre)
forest_f1_score_TNM = f1_score(y_test_TNM,forest_y_pre)
forest_auc_TNM=roc_auc_score(y_test_TNM,forest_y_proba_TNM[:,1])

precision_TNM, recall_TNM, _ = precision_recall_curve(y_test_TNM,forest_y_pre)
PRC_TNM = average_precision_score(y_test_TNM,forest_y_pre)

print('forest_accuracy_score: %f,forest_preci_score: %f,forest_recall_score: %f,forest_f1_score: %f,forest_auc_TNM: %f'
      %(forest_accuracy_score,forest_preci_score_TNM,forest_recall_score_TNM,forest_f1_score_TNM,forest_auc_TNM))

# train
forest_y_pre_train=forest_TNM.predict(X_train_TNM)
forest_y_proba_TNM_train=forest_TNM.predict_proba(X_train_TNM)

forest_accuracy_score_train = accuracy_score(y_train_TNM,forest_y_pre_train)
forest_preci_score_TNM_train = precision_score(y_train_TNM,forest_y_pre_train)
forest_recall_score_TNM_train = recall_score(y_train_TNM,forest_y_pre_train)
forest_f1_score_TNM_train = f1_score(y_train_TNM,forest_y_pre_train)
forest_auc_TNM_train = roc_auc_score(y_train_TNM,forest_y_proba_TNM_train[:,1])

precision_TNM_train, recall_TNM_train, _ = precision_recall_curve(y_train_TNM,forest_y_pre_train)
PRC_TNM_train = average_precision_score(y_train_TNM,forest_y_pre_train)

print('forest_accuracy_score_train: %f,forest_preci_score_train: %f,forest_recall_score_train: %f,forest_f1_score_train: %f,forest_auc_TNM_train: %f'
      %(forest_accuracy_score_train,forest_preci_score_TNM_train,forest_recall_score_TNM_train,forest_f1_score_TNM_train,forest_auc_TNM_train))

# validation
forest_y_pre_validation=forest_TNM.predict(X_validation_TNM)
forest_y_proba_TNM_validation=forest_TNM.predict_proba(X_validation_TNM)

forest_accuracy_score_validation = accuracy_score(y_validation_TNM,forest_y_pre_validation)
forest_preci_score_TNM_validation = precision_score(y_validation_TNM,forest_y_pre_validation)
forest_recall_score_TNM_validation = recall_score(y_validation_TNM,forest_y_pre_validation)
forest_f1_score_TNM_validation = f1_score(y_validation_TNM,forest_y_pre_validation)
forest_auc_TNM_validation = roc_auc_score(y_validation_TNM,forest_y_proba_TNM_validation[:,1])

precision_TNM_validation, recall_TNM_validation, _ = precision_recall_curve(y_validation_TNM,forest_y_pre_validation)
PRC_TNM_validation = average_precision_score(y_validation_TNM,forest_y_pre_validation)

print('forest_accuracy_score_validation: %f,forest_preci_score_validation: %f,forest_recall_score_validation: %f,forest_f1_score_validation: %f,forest_auc_TNM_validation: %f'
      %(forest_accuracy_score_validation,forest_preci_score_TNM_validation,forest_recall_score_TNM_validation,forest_f1_score_TNM_validation,forest_auc_TNM_validation))

print('-----------------------------------------------------------------------','\n')



## 计算性能

In [ ]:
import seaborn as sns
# test
forest_fpr_gtv_clinical,forest_tpr_gtv_clinical,forest_threasholds_gtv_clinical = roc_curve(y_test_gtv_clinical,forest_y_proba_gtv_clinical[:,1]) # 计算ROC的值,svm_threasholds为阈值
forest_fpr_gtv,forest_tpr_gtv,forest_threasholds_gtv = roc_curve(y_test_gtv,forest_y_proba_gtv[:,1]) # 计算ROC的值,svm_threasholds为阈值

forest_fpr_ctv_clinical,forest_tpr_ctv_clinical,forest_threasholds_ctv_clinical=roc_curve(y_test_ctv_clinical,forest_y_proba_ctv_clinical[:,1]) # 计算ROC的值,svm_threasholds为阈值
forest_fpr_ctv,forest_tpr_ctv,forest_threasholds_ctv=roc_curve(y_test_ctv,forest_y_proba_ctv[:,1]) # 计算ROC的值,svm_threasholds为阈值

forest_fpr_clinical,forest_tpr_clinical,forest_threasholds_clinical=roc_curve(y_test_clinical,forest_y_proba_clinical[:,1]) # 计算ROC的值,svm_threasholds为阈值
forest_fpr_TNM,forest_tpr_TNM,forest_threasholds_TNM=roc_curve(y_test_TNM,forest_y_proba_TNM[:,1]) # 计算ROC的值,svm_threasholds为阈值

#train
forest_fpr_gtv_clinical_train,forest_tpr_gtv_clinical_train,forest_threasholds_gtv_clinical_train=roc_curve(y_train_gtv_clinical,forest_y_proba_gtv_clinical_train[:,1]) # 计算ROC的值,svm_threasholds为阈值
forest_fpr_gtv_train,forest_tpr_gtv_train,forest_threasholds_gtv_train=roc_curve(y_train_gtv,forest_y_proba_gtv_train[:,1]) # 计算ROC的值,svm_threasholds为阈值

forest_fpr_ctv_clinical_train,forest_tpr_ctv_clinical_train,forest_threasholds_ctv_clinical_train=roc_curve(y_train_ctv_clinical,forest_y_proba_ctv_clinical_train[:,1]) # 计算ROC的值,svm_threasholds为阈值
forest_fpr_ctv_train,forest_tpr_ctv_train,forest_threasholds_ctv_train=roc_curve(y_train_ctv,forest_y_proba_ctv_train[:,1]) # 计算ROC的值,svm_threasholds为阈值

forest_fpr_clinical_train,forest_tpr_clinical_train,forest_threasholds_clinical_train=roc_curve(y_train_clinical,forest_y_proba_clinical_train[:,1]) # 计算ROC的值,svm_threasholds为阈值
forest_fpr_TNM_train,forest_tpr_TNM_train,forest_threasholds_TNM_train=roc_curve(y_train_TNM,forest_y_proba_TNM_train[:,1]) # 计算ROC的值,svm_threasholds为阈值

#validation
forest_fpr_gtv_clinical_validation,forest_tpr_gtv_clinical_validation,forest_threasholds_gtv_clinical_validation=roc_curve(y_validation_gtv_clinical,forest_y_proba_gtv_clinical_validation[:,1]) # 计算ROC的值,svm_threasholds为阈值
forest_fpr_gtv_validation,forest_tpr_gtv_validation,forest_threasholds_gtv_validation=roc_curve(y_validation_gtv,forest_y_proba_gtv_validation[:,1]) # 计算ROC的值,svm_threasholds为阈值

forest_fpr_ctv_clinical_validation,forest_tpr_ctv_clinical_validation,forest_threasholds_ctv_clinical_validation=roc_curve(y_validation_ctv_clinical,forest_y_proba_ctv_clinical_validation[:,1]) # 计算ROC的值,svm_threasholds为阈值
forest_fpr_ctv_validation,forest_tpr_ctv_validation,forest_threasholds_ctv_validation=roc_curve(y_validation_ctv,forest_y_proba_ctv_validation[:,1]) # 计算ROC的值,svm_threasholds为阈值

forest_fpr_clinical_validation,forest_tpr_clinical_validation,forest_threasholds_clinical_validation=roc_curve(y_validation_clinical,forest_y_proba_clinical_validation[:,1]) # 计算ROC的值,svm_threasholds为阈值
forest_fpr_TNM_validation,forest_tpr_TNM_validation,forest_threasholds_TNM_validation=roc_curve(y_validation_TNM,forest_y_proba_TNM_validation[:,1]) # 计算ROC的值,svm_threasholds为阈值


### 95% confidence intervals for AUCs

In [ ]:
import numpy as np
from sklearn.utils import resample
from sklearn.metrics import roc_auc_score

def compute_auc_ci(y_true, y_pred, n_bootstraps=1000, ci=95):
    """
    Calculation of AUC for binary classification models and confidence intervals for AUC using the self-help method.
    
    Parameters:
    y_true: array-like, true labels for binary classification, shape (n_samples,).
    y_pred: array-like, predicted label probability for binary classification, shape (n_samples,).
    n_bootstraps: int, number of repeated samples for self-help method. Default is 1000.
    ci: int, the percentage of confidence intervals to be calculated. Defaults to 95.
    
    return:
    The function returns a tuple with the AUC and the upper and lower bounds of the confidence interval.
    """
    aucs = []
    for i in range(n_bootstraps):
        y_pred_bootstrap, y_true_bootstrap = resample(y_pred, y_true)
        if len(np.unique(y_true_bootstrap)) < 2:
            continue
        aucs.append(roc_auc_score(y_true_bootstrap, y_pred_bootstrap))
    lower = (100 - ci) / 2.0
    upper = ci + lower
    auc_mean = round(np.mean(aucs), 2)
    auc_lower = round(np.percentile(aucs, lower), 2)
    auc_upper = round(np.percentile(aucs, upper), 2)
    return auc_mean, [auc_lower,auc_upper]


In [ ]:
print("gtv_clinical")
roc_auc_gtv_clinical, CI_gtv_clinical = compute_auc_ci(y_test_gtv_clinical,forest_y_proba_gtv_clinical[:,1])
print(f"AUC = ", roc_auc_gtv_clinical)
print(f"95% Confidence Interval: ", CI_gtv_clinical)
roc_auc_gtv_clinical_train, CI_gtv_clinical_train = compute_auc_ci(y_train_gtv_clinical,forest_y_proba_gtv_clinical_train[:,1])
print(f"AUC = ", roc_auc_gtv_clinical_train)
print(f"95% Confidence Interval: ", CI_gtv_clinical_train)
roc_auc_gtv_clinical_validation, CI_gtv_clinical_validation = compute_auc_ci(y_validation_gtv_clinical,forest_y_proba_gtv_clinical_validation[:,1])
print(f"AUC = ", roc_auc_gtv_clinical_validation)
print(f"95% Confidence Interval: ", CI_gtv_clinical_validation)

print("\ngtv")
roc_auc_gtv, CI_gtv = compute_auc_ci(y_test_gtv,forest_y_proba_gtv[:,1])
print(f"AUC = ", roc_auc_gtv)
print(f"95% Confidence Interval: ", CI_gtv)
roc_auc_gtv_train, CI_gtv_train = compute_auc_ci(y_train_gtv,forest_y_proba_gtv_train[:,1])
print(f"AUC = ", roc_auc_gtv_train)
print(f"95% Confidence Interval: ", CI_gtv_train)
roc_auc_gtv_validation, CI_gtv_validation = compute_auc_ci(y_validation_gtv,forest_y_proba_gtv_validation[:,1])
print(f"AUC = ", roc_auc_gtv_validation)
print(f"95% Confidence Interval: ", CI_gtv_validation)

print("\nctv_clinical")
roc_auc_ctv_clinical, CI_ctv_clinical = compute_auc_ci(y_test_ctv_clinical,forest_y_proba_ctv_clinical[:,1])
print(f"AUC = ", roc_auc_ctv_clinical)
print(f"95% Confidence Interval: ", CI_ctv_clinical)
roc_auc_ctv_clinical_train, CI_ctv_clinical_train = compute_auc_ci(y_train_ctv_clinical,forest_y_proba_ctv_clinical_train[:,1])
print(f"AUC = ", roc_auc_ctv_clinical_train)
print(f"95% Confidence Interval: ", CI_ctv_clinical_train)
roc_auc_ctv_clinical_validation, CI_ctv_clinical_validation = compute_auc_ci(y_validation_ctv_clinical,forest_y_proba_ctv_clinical_validation[:,1])
print(f"AUC = ", roc_auc_ctv_clinical_validation)
print(f"95% Confidence Interval: ", CI_ctv_clinical_validation)

print("\nctv")
roc_auc_ctv, CI_ctv = compute_auc_ci(y_test_ctv,forest_y_proba_ctv[:,1])
print(f"AUC = ", roc_auc_ctv)
print(f"95% Confidence Interval: ", CI_ctv)
roc_auc_ctv_train, CI_ctv_train = compute_auc_ci(y_train_ctv,forest_y_proba_ctv_train[:,1])
print(f"AUC = ", roc_auc_ctv_train)
print(f"95% Confidence Interval: ", CI_ctv_train)
roc_auc_ctv_validation, CI_ctv_validation = compute_auc_ci(y_validation_ctv,forest_y_proba_ctv_validation[:,1])
print(f"AUC = ", roc_auc_ctv_validation)
print(f"95% Confidence Interval: ", CI_ctv_validation)

print("\nclinical")
roc_auc_clinical, CI_clinical = compute_auc_ci(y_test_clinical,forest_y_proba_clinical[:,1])
print(f"AUC = ", roc_auc_clinical)
print(f"95% Confidence Interval: ", CI_clinical)
roc_auc_clinical_train, CI_clinical_train = compute_auc_ci(y_train_clinical,forest_y_proba_clinical_train[:,1])
print(f"AUC = ", roc_auc_clinical_train)
print(f"95% Confidence Interval: ", CI_clinical_train)
roc_auc_clinical_validation, CI_clinical_validation = compute_auc_ci(y_validation_clinical,forest_y_proba_clinical_validation[:,1])
print(f"AUC = ", roc_auc_clinical_validation)
print(f"95% Confidence Interval: ", CI_clinical_validation)

print("\nTNM")
roc_auc_TNM, CI_TNM = compute_auc_ci(y_test_TNM,forest_y_proba_TNM[:,1])
print(f"AUC = ", roc_auc_TNM)
print(f"95% Confidence Interval: ", CI_TNM)
roc_auc_TNM_train, CI_TNM_train = compute_auc_ci(y_train_TNM,forest_y_proba_TNM_train[:,1])
print(f"AUC = ", roc_auc_TNM_train)
print(f"95% Confidence Interval: ", CI_TNM_train)
roc_auc_TNM_validation, CI_TNM_validation = compute_auc_ci(y_validation_TNM,forest_y_proba_TNM_validation[:,1])
print(f"AUC = ", roc_auc_TNM_validation)
print(f"95% Confidence Interval: ", CI_TNM_validation)


In [ ]:
import pandas as pd
from scipy.stats import shapiro, wilcoxon
from scipy.stats import mannwhitneyu
def wilcoxon_signed_rank_test(df, col1, col2):
    '''
    Performs a significance test on specified columns of two classifier models on the same dataset using the Wilcoxon signed-rank test and outputs the test results.
    df: DataFrame containing the two columns of data to be tested for significance
    col1: str, column name of the first column of data
    col2: str, column name of the second column of data
    '''
    stat1, p1 = shapiro(df[col1])
    stat2, p2 = shapiro(df[col2])

    if p1 < 0.05 or p2 < 0.05:
#         print('The %s data for Classifier 1 or Classifier 2 does not conform to a normal distribution and the Mann-Whitney U test will be used'' % (col1))
        stat, p = mannwhitneyu(df[col1], df[col2])
    else:
#         print('The %s data for classifier 1 and classifier 2 conforms to a normal distribution and will be tested using the Wilcoxon signed-rank test' % (col1))
        diff = df[col1] - df[col2]
        stat, p = wilcoxon(diff)
    print('Wilcoxon signed-rank test：Statistic=%.3f, p=%.3f' % (stat, p))
    return p


In [ ]:
from scipy.stats import friedmanchisquare

auc_list = [forest_y_proba_gtv_clinical[:,1],
            forest_y_proba_gtv[:,1], 
            forest_y_proba_ctv_clinical[:,1], 
            forest_y_proba_ctv[:,1], 
            forest_y_proba_clinical[:,1]
           ]
stat, p_test = friedmanchisquare(*auc_list)
print('Friedman Chi-Squared Statistic: %.3f' % stat)
print('p-value: %.4f' % p_test)
    
auc_list = [forest_y_proba_gtv_clinical_train[:,1],
            forest_y_proba_gtv_train[:,1], 
            forest_y_proba_ctv_clinical_train[:,1], 
            forest_y_proba_ctv_train[:,1], 
            forest_y_proba_clinical_train[:,1]
           ]
stat, p_train = friedmanchisquare(*auc_list)
print('Friedman Chi-Squared Statistic: %.3f' % stat)
print('p-value: %.4f' % p_train)
    
auc_list = [forest_y_proba_gtv_clinical_validation[:,1],
            forest_y_proba_gtv_validation[:,1], 
            forest_y_proba_ctv_clinical_validation[:,1], 
            forest_y_proba_ctv_validation[:,1], 
            forest_y_proba_clinical_validation[:,1]
           ]
stat, p_validation = friedmanchisquare(*auc_list)
print('Friedman Chi-Squared Statistic: %.3f' % stat)
print('p-value: %.4f' % p_validation)

    
print("\nTNM")
auc_df = pd.DataFrame({'col_1': forest_y_proba_gtv_clinical[:,1], 
                       'col_2': forest_y_proba_TNM[:,1]})
p_TNM_test = wilcoxon_signed_rank_test(auc_df, 'col_1', 'col_2')
print(np.round(p_TNM_test, 4))

auc_df = pd.DataFrame({'col_1': forest_y_proba_gtv_clinical_train[:,1], 
                       'col_2': forest_y_proba_TNM_train[:,1]})
p_TNM_train = wilcoxon_signed_rank_test(auc_df, 'col_1', 'col_2')
print(np.round(p_TNM_train, 4))

auc_df = pd.DataFrame({'col_1': forest_y_proba_gtv_clinical_validation[:,1], 
                       'col_2': forest_y_proba_TNM_validation[:,1]})
p_TNM_validation = wilcoxon_signed_rank_test(auc_df, 'col_1', 'col_2')
print(np.round(p_TNM_validation, 4))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['font.family'] = 'Times New Roman'
fig, axs = plt.subplots(2, 3, figsize=(18, 12))
lw = 2
axs[0, 0].plot(forest_fpr_clinical_train, forest_tpr_clinical_train, color = '#7B7879', lw=lw, label="clinical (%0.2f)" % roc_auc_clinical_train)
axs[0, 0].plot(forest_fpr_ctv_train, forest_tpr_ctv_train, color = '#EBA716', lw=lw, label="CTV (%0.2f)" % roc_auc_ctv_train)
axs[0, 0].plot(forest_fpr_gtv_train, forest_tpr_gtv_train, color = '#64B42A', lw=lw, label="GTV (%0.2f)" % roc_auc_gtv_train)
axs[0, 0].plot(forest_fpr_ctv_clinical_train, forest_tpr_ctv_clinical_train, color = '#6E208F', lw=lw, label="CTV_Clinical (%0.2f)" % roc_auc_ctv_clinical_train)
axs[0, 0].plot(forest_fpr_gtv_clinical_train, forest_tpr_gtv_clinical_train, color = '#E1491A', lw=lw, label="GTV_Clinical (0.99)")
axs[0, 0].set_xlabel("False Positive Rate", fontsize=29)
axs[0, 0].set_ylabel("True Positive Rate", fontsize=29)
axs[0, 0].plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
axs[0, 0].set_xlim(-0.02,1.03)
axs[0, 0].set_ylim(-0.02,1.03)
axs[0, 0].tick_params(axis='x', labelsize=20, width=2, length=4)
axs[0, 0].tick_params(axis='y', labelsize=20, width=2, length=4)
axs[0, 0].set_title("ROC (training set)", fontsize=30)
axs[0, 0].legend(loc="lower right", fontsize=15, framealpha=0 )

axs[0, 1].plot(forest_fpr_clinical, forest_tpr_clinical, color = '#7B7879', lw=lw, label="clinical (%0.2f)" % roc_auc_clinical)
axs[0, 1].plot(forest_fpr_ctv, forest_tpr_ctv, color = '#EBA716', lw=lw, label="CTV (%0.2f)" % roc_auc_ctv)
axs[0, 1].plot(forest_fpr_gtv, forest_tpr_gtv, color = '#64B42A', lw=lw, label="GTV (%0.2f)" % roc_auc_gtv)
axs[0, 1].plot(forest_fpr_ctv_clinical, forest_tpr_ctv_clinical, color = '#6E208F', lw=lw, label="CTV_Clinical (%0.2f)" % roc_auc_ctv_clinical)
axs[0, 1].plot(forest_fpr_gtv_clinical, forest_tpr_gtv_clinical, color='#E1491A', lw=lw, label="GTV_Clinical (%0.2f)" % roc_auc_gtv_clinical)

axs[0, 1].set_xlabel("False Positive Rate", fontsize=29)
axs[0, 1].set_ylabel("True Positive Rate", fontsize=29)
axs[0, 1].plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
axs[0, 1].set_xlim(-0.02,1.03)
axs[0, 1].set_ylim(-0.02,1.03)
axs[0, 1].tick_params(axis='x', labelsize=20, width=2, length=4)
axs[0, 1].tick_params(axis='y', labelsize=20, width=2, length=4)
axs[0, 1].set_title("ROC (test set)", fontsize=30)
axs[0, 1].legend(loc="lower right", fontsize=14,framealpha=0 )

axs[0, 2].plot(forest_fpr_clinical_validation, forest_tpr_clinical_validation, color = '#7B7879', lw=lw, label="clinical (%0.2f)" % roc_auc_clinical_validation)
axs[0, 2].plot(forest_fpr_ctv_validation, forest_tpr_ctv_validation, color = '#EBA716', lw=lw, label="CTV (%0.2f)" % roc_auc_ctv_validation)
axs[0, 2].plot(forest_fpr_gtv_validation, forest_tpr_gtv_validation, color = '#64B42A', lw=lw, label="GTV (%0.2f)" % roc_auc_gtv_validation)
axs[0, 2].plot(forest_fpr_ctv_clinical_validation, forest_tpr_ctv_clinical_validation, color = '#6E208F', lw=lw, label="CTV_Clinical (%0.2f)" % roc_auc_ctv_clinical_validation)
axs[0, 2].plot(forest_fpr_gtv_clinical_validation, forest_tpr_gtv_clinical_validation, color = '#E1491A', lw=lw, label="GTV_Clinical (%0.2f)" % roc_auc_gtv_clinical_validation)
axs[0, 2].set_xlabel("False Positive Rate", fontsize=29)
axs[0, 2].set_ylabel("True Positive Rate", fontsize=29)
axs[0, 2].plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
axs[0, 2].set_xlim(-0.02,1.03)
axs[0, 2].set_ylim(-0.02,1.03)
axs[0, 2].tick_params(axis='x', labelsize=20, width=2, length=4)
axs[0, 2].tick_params(axis='y', labelsize=20, width=2, length=4)
axs[0, 2].set_title("ROC (validation set)", fontsize=30)
axs[0, 2].legend(loc="lower right", fontsize=14, framealpha=0 )

axs[1, 0].plot(forest_fpr_TNM_train, forest_tpr_TNM_train, color = '#0000FF', lw=lw, label="TNM (%0.2f)" % roc_auc_TNM_train)
axs[1, 0].plot(forest_fpr_gtv_clinical_train, forest_tpr_gtv_clinical_train, color = '#E1491A', lw=lw, label="GTV_Clinical (0.99)")
axs[1, 0].set_xlabel("False Positive Rate", fontsize=29)
axs[1, 0].set_ylabel("True Positive Rate", fontsize=29)
axs[1, 0].plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
axs[1, 0].set_xlim(-0.02,1.03)
axs[1, 0].set_ylim(-0.02,1.03)
axs[1, 0].tick_params(axis='x', labelsize=20, width=2, length=4)
axs[1, 0].tick_params(axis='y', labelsize=20, width=2, length=4)
axs[1, 0].set_title("ROC (training set)", fontsize=30)
axs[1, 0].legend(loc="lower right", fontsize=14, framealpha=0 )

axs[1, 1].plot(forest_fpr_TNM, forest_tpr_TNM, color = '#0000FF', lw=lw, label="TNM (%0.2f)" % roc_auc_TNM)
axs[1, 1].plot(forest_fpr_gtv_clinical, forest_tpr_gtv_clinical, color = '#E1491A', lw=lw, label="GTV_Clinical (%0.2f)" % roc_auc_gtv_clinical)
axs[1, 1].set_xlabel("False Positive Rate", fontsize=29)
axs[1, 1].set_ylabel("True Positive Rate", fontsize=29)
axs[1, 1].plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
axs[1, 1].set_xlim(-0.02,1.03)
axs[1, 1].set_ylim(-0.02,1.03)
axs[1, 1].tick_params(axis='x', labelsize=20, width=2, length=4)
axs[1, 1].tick_params(axis='y', labelsize=20, width=2, length=4)
axs[1, 1].set_title("ROC (test set)", fontsize=30)
axs[1, 1].legend(loc="lower right", fontsize=14, framealpha=0 )

axs[1, 2].plot(forest_fpr_TNM_validation, forest_tpr_TNM_validation, color = '#0000FF', lw=lw, label="TNM (%0.2f)" % roc_auc_TNM_validation)
axs[1, 2].plot(forest_fpr_gtv_clinical_validation, forest_tpr_gtv_clinical_validation, color = '#E1491A', lw=lw, label="GTV_Clinical (%0.2f)" % roc_auc_gtv_clinical_validation)
axs[1, 2].set_xlabel("False Positive Rate", fontsize=29)
axs[1, 2].set_ylabel("True Positive Rate", fontsize=29)
axs[1, 2].plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
axs[1, 2].set_xlim(-0.02,1.03)
axs[1, 2].set_ylim(-0.02,1.03)
axs[1, 2].tick_params(axis='x', labelsize=20, width=2, length=4)
axs[1, 2].tick_params(axis='y', labelsize=20, width=2, length=4)
axs[1, 2].set_title("ROC (validation set)", fontsize=30)
axs[1, 2].legend(loc="lower right", fontsize=14, framealpha=0 )

axs[0, 0].annotate("a.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')
axs[0, 1].annotate("b.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')
axs[0, 2].annotate("c.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')
axs[1, 0].annotate("d.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')
axs[1, 1].annotate("e.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')
axs[1, 2].annotate("f.", (-0.22, 1.07), xycoords='axes fraction', fontsize=30, fontweight='bold', va='top')

axs[0, 0].annotate("p = "+str(np.round(p_train, 2)), (0.66, 0.62), xycoords='axes fraction', fontsize=20, va='top')
axs[0, 1].annotate("p = "+str(np.round(p_test, 2)), (0.66, 0.62), xycoords='axes fraction', fontsize=20, va='top')
axs[0, 2].annotate("p = "+str(np.round(p_validation, 2)), (0.66, 0.62), xycoords='axes fraction', fontsize=20, va='top')
axs[1, 0].annotate("p = "+str(np.round(p_TNM_train, 2)), (0.66, 0.52), xycoords='axes fraction', fontsize=20, va='top')
axs[1, 1].annotate("p = "+str(np.round(p_TNM_test, 2)), (0.66, 0.52), xycoords='axes fraction', fontsize=20, va='top')
axs[1, 2].annotate("p = "+str(np.round(p_TNM_validation, 2)), (0.66, 0.52), xycoords='axes fraction', fontsize=20, va='top')

fig.subplots_adjust(hspace=0.35, wspace=0.30, top=0.85, bottom=0.1, left=0.08, right=0.92)

# plt.savefig(".//figure4_ROC.tif", dpi=300)

plt.show()


## SHAP

In [ ]:
shap.initjs()
explainer = shap.KernelExplainer(forest_gtv_clinical.predict_proba, X_test_gtv_clinical)
shap_values = explainer.shap_values(X_test_gtv_clinical)

In [ ]:
shap.summary_plot(shap_values[0], X_test_gtv_clinical, show=False, color_bar=False)

plt.xticks(fontsize=16)
plt.yticks(fontsize=19)
font2 = {'family' : 'Times New Roman',
    'weight' : 'normal',
    'size' : 20,
    }
plt.xlabel('SHAP value (impact on model output)',font2)
cb = plt.colorbar(fraction=0.2,ticks=[-1, 1], aspect=80)
cb.set_ticklabels(["low", "high"])
cb.set_label("Feature Value", size=19, labelpad=0)
cb.ax.tick_params(labelsize=15, length=0)
cb.set_alpha(1)
cb.outline.set_visible(False)

In [ ]:
shap.summary_plot(shap_values[1], X_test_gtv_clinical, show=False, color_bar=False)

plt.xticks(fontsize=16)
plt.yticks(fontsize=19)
font2 = {'family' : 'Times New Roman',
    'weight' : 'normal',
    'size' : 20,
    }
plt.xlabel('SHAP value (impact on model output)',font2)
cb = plt.colorbar(fraction=0.2,ticks=[-1, 1], aspect=80)
cb.set_ticklabels(["low", "high"])
cb.set_label("Feature Value", size=19, labelpad=0)
cb.ax.tick_params(labelsize=15, length=0)
cb.set_alpha(1)
cb.outline.set_visible(False)
plt.show()

In [ ]:
shap.summary_plot(shap_values, X_test_gtv_clinical, show=False, color_bar=False)

plt.xticks(fontsize=16)
plt.yticks(fontsize=19)
font2 = {'family' : 'Times New Roman',
    'weight' : 'normal',
    'size' : 18,
    }
plt.xlabel('mean(|SHAP value|) (average impact on model output magnitude)',font2)
plt.show()

In [ ]:
explainerTR = shap.TreeExplainer(forest_gtv_clinical)
shap_interaction_values = explainerTR.shap_interaction_values(X_test_gtv_clinical)
shap.summary_plot(shap_interaction_values[1], X_test_gtv_clinical, show=False)

plt.xticks(fontsize=17)

plt.show()

In [ ]:
shap_values=np.array(shap_values)
shap.force_plot(explainer.expected_value[1], shap_values[1,1], feature_names=X_test_gtv_clinical.columns.tolist(),matplotlib=True,show=False)
plt.xticks(fontsize=20)
plt.text(0.653, 0.405, 'higher', fontsize=20, color='#FF0D57', horizontalalignment='right')
plt.text(0.657, 0.405, 'lower', fontsize=20, color='#1E88E5', horizontalalignment='left')
plt.text(0.655, 0.4, r'$\leftarrow$', fontsize=20, color='#1E88E5', horizontalalignment='center')
plt.text(0.655, 0.425, r'$\rightarrow$',fontsize=20, color='#FF0D57',horizontalalignment='center')

text_out_val = plt.text(0.643, 0.33, 'base value', fontsize=20, alpha=0.5, horizontalalignment='center')
text_out_val.set_bbox(dict(facecolor='white', edgecolor='white'))

text_out_val = plt.text(0.593, 0.33, 'f(x)', fontsize=18, alpha=0.5, horizontalalignment='center')
text_out_val.set_bbox(dict(facecolor='white', edgecolor='white'))

plt.show()

## Pearson's correlation coefficient

In [ ]:
new_df=X_test_gtv_clinical.corr()
import seaborn as sns  
plt.figure(1)
sns.heatmap(new_df,annot=True, vmax=1, square=True, cmap='RdBu_r',center=0.8)
plt.show()